<img src="https://carboncopy.info/wp-content/uploads/air-pollution.jpeg" length="100px" width="1000px">

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/india-air-quality-data/data.csv',encoding='cp1252')
df.head()

In [ ]:
df.shape

# We are droping the following columns
* **stn_code:** Just a code and it will not gonna help fot the analysis
* **sampling_data:** We have another datcolumn with clear format
* **spm:** Missing values are more than 50%
* **pm2_5:** Missing values are more than 50%

In [ ]:
df.drop(['stn_code','sampling_date', 'spm', 'pm2_5'], axis=1, inplace=True)

In [ ]:
#show the information about dataset
df_dtype=df.dtypes
df_missing=df.isna().mean().round(5)*100
df_desc = df.describe(include='all')
dtls_1=pd.DataFrame(df_missing, columns=['missing'])
dtls_2=pd.DataFrame(df_dtype, columns=['dtype'])
dtls_3=pd.DataFrame(df_desc)
dtls_3=dtls_3.T
dtls=pd.concat([dtls_1, dtls_2, dtls_3], axis=1)
dtls.sort_values(by='missing', ascending=False)

In [ ]:
for i in list(df.select_dtypes(include='float64')):
    df[i].fillna(df[i].mean(), inplace=True)
    
df.isna().mean().round(5)*100

In [ ]:
#imputing null values in agencies with unknown is the correct way. If we are not sure about the name of the agencies, we should not impute with mode or other things

df['agency'].fillna('unknown', inplace=True)
df['location_monitoring_station'].fillna('unknown', inplace=True)

# though the null values are less in type feature we will use mode to impute those NAs
df['type'].fillna(df.type.mode()[0], inplace=True)
df['location'].fillna(df.location.mode()[0], inplace=True)
df['date'].fillna(df.date.mode()[0], inplace=True)

df.isna().sum().sum()



In [ ]:
df.type.value_counts()

In [ ]:
df['type']=df['type'].replace(('Residential and others'),('Residential'))
df['type']=df['type'].replace(('Residential, Rural and other Areas'),('RIRUO'))
df['type']=df['type'].replace(('Industrial Area', 'Industrial Areas'),('Industrial', 'Industrial'))
df['type']=df['type'].replace(('Sensitive Area', 'Sensitive Areas'),('Sensitive', 'Sensitive'))

df.type.value_counts()

In [ ]:
df.location.value_counts()

* Respirable suspended particulate matter (RSPM), the deadliest pollutant in the city air.
* Let's analyze which state is worst in case of RSPM

In [ ]:
#top 10 worst states on the basis of rspm
df[['state', 'rspm']].groupby(['state']).agg('mean').sort_values('rspm',
                                                                 ascending=False).head(10).style.background_gradient(cmap='viridis')

In [ ]:
#top 10 worst cities on the basis of rspm

df[['location', 'rspm']].groupby(['location']).agg('mean').sort_values('rspm',
                                                                 ascending=False).head(10).style.background_gradient(cmap='viridis')

In [ ]:
#Let's analyze which type of areas are mostly polluted

df[['type', 'rspm']].groupby(['type']).agg('mean').sort_values('rspm', ascending=False).style.background_gradient(cmap='inferno')

# Let's analyze in Tamil Nadu

In [ ]:
df[df['state']=='Tamil Nadu']['location'].unique()

In [ ]:
df['location']=df['location'].replace(('Madras', 'Turicorin', 'Thoothukudi', 'Mettur'),('Chennai', 'Tuticorin', 'Tuticorin', 'Salem'))

In [ ]:
df[df['state']=='Tamil Nadu'][['location', 'rspm']].groupby(['location']).agg('mean').sort_values('rspm',
                                                                                                  ascending=False).style.background_gradient(cmap='cool')

In [ ]:
#let's check which type of area causing more pollution in tuticorin
df[df['location']=='Tuticorin'][['type', 'rspm']].groupby(['type']).agg('mean').sort_values('rspm',
                                                                                           ascending=False).style.background_gradient(cmap='inferno')

In [ ]:
#As we know Industrial areas causing more pollution let's check which industrial areas causing more pollution in tamilnadu
df[(df['type']=='industrial')|(df['state']=='Tamil Nadu')][['location', 
                                                            'rspm']].groupby(['location']).agg('mean').sort_values('rspm', 
                                                                                                                   ascending=False).style.background_gradient(cmap='rainbow')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sorted=df.groupby(['state'])['so2'].agg('mean').reset_index().sort_values('so2', ascending=False).head(10)
plt.rcParams['figure.figsize']=6,7
sns.barplot(y=df['state'], x=df['so2'], palette='plasma', order=sorted['state'])
plt.show()

In [ ]:
sorted=df.groupby(['state'])['no2'].agg('mean').reset_index().sort_values('no2', ascending=False).head(10)
plt.rcParams['figure.figsize']=6,7
sns.barplot(y=df['state'], x=df['no2'], order=sorted['state'])
plt.show()

In [ ]:
sorted=df.groupby(['state'])['rspm'].agg('mean').reset_index().sort_values('rspm', ascending=False).head(10)
plt.rcParams['figure.figsize']=6,7
sns.barplot(y=df['state'], x=df['rspm'], order=sorted['state'], palette='cool')
plt.show()